In [1]:
from rdflib_r2r import R2RStore, R2RMapping
from sqlalchemy import create_engine
from pathlib import Path
import rdflib
from rdflib.compare import to_isomorphic, graph_diff

echo = False


def create_database(db, path, *sql_fnames):
    for sql_fname in sql_fnames:
        sql_script = (path / sql_fname).open().read()
        conn = db.raw_connection()
        try:
            if hasattr(conn.connection, "c"):
                cursor = conn.connection.c
            else:
                cursor = conn.cursor()
            cursor.execute(sql_script, (), None)
        except Exception as e:
            try:
                cursor = conn.cursor()
                cursor.executescript(sql_script)
            except Exception as e:
                raise Exception(f"Problem with {path}: {e}")
            finally:
                conn.close()
        finally:
            conn.close()

# db = create_engine(
#     "duckdb:///:memory:",
#     echo=echo,
#     # future=True,
#     connect_args={"read_only": False},
# )
# db.dialect.server_version_info = (0,)

import sqlite3
db = create_engine(
    "sqlite:///:memory:",
    echo=echo,
    future=True,
    connect_args={
        "detect_types": sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
        'timeout': 30 * 1000
    },
    native_datetime=False, # breaks rdb2rdf test D016
    # native_datetime=True, # breaks BSBM test 3
)

path = Path('tests/rdb2rdf-ts/D014-3tables1primarykey1foreignkey')
sql_fname = 'create.sql'

sql_script = (path / sql_fname).open().read()
create_database(db, path, sql_fname)


# mapfile = path / 'r2rmla.ttl'
# fmt = rdflib.util.guess_format(str(mapfile))
# mapping = R2RMapping(rdflib.Graph().parse(str(mapfile), format=fmt))

mapping = R2RMapping.from_db(db)

g_made = rdflib.Graph(R2RStore(db=db, mapping=mapping))

print(g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}"))

@base <http://example.com/base/> .
@prefix ns1: <http://www.w3.org/ns/r2rml#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a ns1:TriplesMap ;
    ns1:logicalTable [ ns1:tableName "\"LIKES\"" ] ;
    ns1:predicateObjectMap [ ns1:objectMap [ ns1:column "\"id\"" ;
                    ns1:datatype xsd:integer ] ;
            ns1:predicate <LIKES#id> ],
        [ ns1:objectMap [ ns1:column "\"likeType\"" ] ;
            ns1:predicate <LIKES#likeType> ],
        [ ns1:objectMap [ ns1:column "\"likedObj\"" ] ;
            ns1:predicate <LIKES#likedObj> ] ;
    ns1:subjectMap [ ns1:class <LIKES> ;
            ns1:termType ns1:BlankNode ] .

[] a ns1:TriplesMap ;
    ns1:logicalTable [ ns1:tableName "\"EMP\"" ] ;
    ns1:predicateObjectMap [ ns1:objectMap [ ns1:column "\"ename\"" ] ;
            ns1:predicate <EMP#ename> ],
        [ ns1:objectMap [ ns1:column "\"deptno\"" ;
                    ns1:datatype xsd:integer ] ;
            ns1:predicate <EMP#deptno> ],
        [ ns1:obje

SELECT anon_1.s AS s,
       anon_1.p AS p,
       anon_1.o AS o
FROM
  (SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
          '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p,
          '<http://example.com/base/DEPT>' AS o,
          NULL AS g
   FROM "DEPT"
   UNION ALL SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
                    '<http://example.com/base/DEPT#dname>' AS p,
                    "DEPT".dname AS o,
                    NULL AS g
   FROM "DEPT"
   UNION ALL SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
                    '<http://example.com/base/DEPT#loc>' AS p,
                    "DEPT".loc AS o,
                    NULL AS g
   FROM "DEPT"
   UNION ALL SELECT CAST('_:DEPT#' AS VARCHAR) || CAST(CAST("DEPT".rowid AS VARCHAR) AS VARCHAR) AS s,
                    '<http://example.com/base/DEPT#deptno>' AS p,
                

In [2]:
from sqlalchemy import text

with db.connect() as conn:
    for r in conn.execute(text(g_made.store.getSQL("SELECT * WHERE {?s ?p ?o}"))):
        print(r)

('_:DEPT#1', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://example.com/base/DEPT>')
('_:DEPT#1', '<http://example.com/base/DEPT#dname>', 'APPSERVER')
('_:DEPT#1', '<http://example.com/base/DEPT#loc>', 'NEW YORK')
('_:DEPT#1', '<http://example.com/base/DEPT#deptno>', '"10"^^<http://www.w3.org/2001/XMLSchema#integer>')
('<EMP/empno=7369>', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://example.com/base/EMP>')
('<EMP/empno=7369>', '<http://example.com/base/EMP#empno>', '"7369"^^<http://www.w3.org/2001/XMLSchema#integer>')
('<EMP/empno=7369>', '<http://example.com/base/EMP#ename>', 'SMITH')
('<EMP/empno=7369>', '<http://example.com/base/EMP#etype>', 'PART_TIME')
('<EMP/empno=7369>', '<http://example.com/base/EMP#job>', 'CLERK')
('<EMP/empno=7369>', '<http://example.com/base/EMP#deptno>', '"10"^^<http://www.w3.org/2001/XMLSchema#integer>')
('<EMP/empno=7369>', '<http://example.com/base/EMP#ref-deptno>', '_:DEPT#1')
('_:LIKES#1', '<http://www.w3.org/1999/02/22-

In [3]:
for t in g_made:
    print(t)

/Users/benno/Documents/postdoc/projects/duckdb-sparql/r2r/rdflib_r2r/r2r_store.py:652: SAWarning: SELECT statement has a cartesian product between FROM element(s) "EMP" and FROM element "DEPT_ref".  Apply join condition(s) between each element to resolve.
  rows = list(conn.execute(query))


(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/DEPT'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#dname'), rdflib.term.Literal('APPSERVER'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#loc'), rdflib.term.Literal('NEW YORK'))
(rdflib.term.BNode('DEPT#1'), rdflib.term.URIRef('http://example.com/base/DEPT#deptno'), rdflib.term.Literal('10', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/EMP'))
(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://example.com/base/EMP#empno'), rdflib.term.Literal('7369', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.te

In [4]:
print(to_isomorphic(g_made).serialize())

@prefix ns1: <http://example.com/base/DEPT#> .
@prefix ns2: <http://example.com/base/LIKES#> .
@prefix ns3: <http://example.com/base/EMP#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/base/EMP/empno=7369> a <http://example.com/base/EMP> ;
    ns3:deptno 10 ;
    ns3:empno 7369 ;
    ns3:ename "SMITH" ;
    ns3:etype "PART_TIME" ;
    ns3:job "CLERK" ;
    ns3:ref-deptno [ a <http://example.com/base/DEPT> ;
            ns1:deptno 10 ;
            ns1:dname "APPSERVER" ;
            ns1:loc "NEW YORK" ] .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Playing" ;
    ns2:likedObj "Soccer" .

[] a <http://example.com/base/LIKES> ;
    ns2:id 7369 ;
    ns2:likeType "Watching" ;
    ns2:likedObj "Basketball" .




In [6]:
outfile = path / 'directGraph.ttl'
# outfile = path / 'mappeda.nq'

fmt = rdflib.util.guess_format(str(outfile))
print(rdflib.Graph().parse(str(outfile), format=fmt).serialize())

for t in rdflib.Graph().parse(str(outfile), format=fmt):
    print(t)

@prefix ns1: <http://example.com/base/LIKES#> .
@prefix ns2: <http://example.com/base/DEPT#> .
@prefix ns3: <http://example.com/base/EMP#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/base/EMP/empno=7369> a <http://example.com/base/EMP> ;
    ns3:deptno 10 ;
    ns3:empno 7369 ;
    ns3:ename "SMITH" ;
    ns3:etype "PART_TIME" ;
    ns3:job "CLERK" ;
    ns3:ref-deptno [ a <http://example.com/base/DEPT> ;
            ns2:deptno 10 ;
            ns2:dname "APPSERVER" ;
            ns2:loc "NEW YORK" ] .

[] a <http://example.com/base/LIKES> ;
    ns1:id 7369 ;
    ns1:likeType "Playing" ;
    ns1:likedObj "Soccer" .

[] a <http://example.com/base/LIKES> ;
    ns1:id 7369 ;
    ns1:likeType "Watching" ;
    ns1:likedObj "Basketball" .


(rdflib.term.URIRef('http://example.com/base/EMP/empno=7369'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://example.com/base/EMP'))
(rdflib.term.BNode('n3255650e940b471c899